In [1]:
import pandas as pd
import numpy as np
import ast
import seaborn as sns
from tqdm import tqdm
from geopy.distance import geodesic
import ml_help
tqdm.pandas()

# Process Gardiner Speed Data

In [2]:
gardiner_speed = pd.read_csv("../../csv/v2/202305_gardiner_flow_data.csv.zip")
gardiner_speed

,frc,currentSpeed,freeFlowSpeed,currentTravelTime,freeFlowTravelTime,confidence,roadClosure,@version,coordinates,collectedAt
0,FRC0,86,92,325,303,1,False,traffic-service-flow 1.0.087,"[{'latitude': 43.681009250673604, 'longitude':...",2023-04-30 20:00:00.007773-04:00
1,FRC0,92,92,303,303,1,False,traffic-service-flow 1.0.087,"[{'latitude': 43.681009250673604, 'longitude':...",2023-04-30 20:05:00.005461-04:00
2,FRC0,92,92,303,303,1,False,traffic-service-flow 1.0.087,"[{'latitude': 43.681009250673604, 'longitude':...",2023-04-30 20:10:00.004552-04:00
3,FRC0,92,92,303,303,1,False,traffic-service-flow 1.0.087,"[{'latitude': 43.681009250673604, 'longitude':...",2023-04-30 20:15:00.009415-04:00
4,FRC0,92,92,303,303,1,False,traffic-service-flow 1.0.087,"[{'latitude': 43.681009250673604, 'longitude':...",2023-04-30 20:20:00.002541-04:00
...,...,...,...,...,...,...,...,...,...,...
8922,FRC0,73,93,382,300,1,False,traffic-service-flow 1.0.088,"[{'latitude': 43.681009250673604, 'longitude':...",2023-05-31 19:35:00.003493-04:00
8923,FRC0,78,93,358,300,1,False,traffic-service-flow 1.0.088,"[{'latitude': 43.681009250673604, 'longitude':...",2023-05-31 19:40:00.004800-04:00
8924,FRC0,78,93,358,300,1,False,traffic-service-flow 1.0.088,"[{'latitude': 43.681009250673604, 'longitude':...",2023-05-31 19:45:00.007860-04:00
8925,FRC0,78,92,358,303,1,False,traffic-service-flow 1.0.088,"[{'latitude': 43.681009250673604, 'longitude':...",2023-05-31 19:50:00.005163-04:00


In [3]:
gardiner_speed.drop(["frc", "freeFlowSpeed", "currentTravelTime", "freeFlowTravelTime", "confidence", "roadClosure", "@version", "coordinates"], axis=1, inplace=True)
gardiner_speed

,currentSpeed,collectedAt
0,86,2023-04-30 20:00:00.007773-04:00
1,92,2023-04-30 20:05:00.005461-04:00
2,92,2023-04-30 20:10:00.004552-04:00
3,92,2023-04-30 20:15:00.009415-04:00
4,92,2023-04-30 20:20:00.002541-04:00
...,...,...
8922,73,2023-05-31 19:35:00.003493-04:00
8923,78,2023-05-31 19:40:00.004800-04:00
8924,78,2023-05-31 19:45:00.007860-04:00
8925,78,2023-05-31 19:50:00.005163-04:00


In [4]:
gardiner_speed["collectedAt"] = pd.to_datetime(gardiner_speed["collectedAt"], utc=True)
gardiner_speed["collectedAt"] = gardiner_speed["collectedAt"].dt.floor(freq="t")
gardiner_speed

,currentSpeed,collectedAt
0,86,2023-05-01 00:00:00+00:00
1,92,2023-05-01 00:05:00+00:00
2,92,2023-05-01 00:10:00+00:00
3,92,2023-05-01 00:15:00+00:00
4,92,2023-05-01 00:20:00+00:00
...,...,...
8922,73,2023-05-31 23:35:00+00:00
8923,78,2023-05-31 23:40:00+00:00
8924,78,2023-05-31 23:45:00+00:00
8925,78,2023-05-31 23:50:00+00:00


# Process Yonge Speed Data

In [5]:
yonge_speed = pd.read_csv("../../csv/v2/202305_yonge_flow_data.csv.zip")
yonge_speed

,frc,currentSpeed,freeFlowSpeed,currentTravelTime,freeFlowTravelTime,confidence,roadClosure,@version,coordinates,collectedAt
0,FRC3,12,17,84,59,1.0,False,traffic-service-flow 1.0.087,"[{'latitude': 43.646886954993356, 'longitude':...",2023-04-30 20:00:00.007773-04:00
1,FRC3,13,18,77,56,1.0,False,traffic-service-flow 1.0.087,"[{'latitude': 43.646886954993356, 'longitude':...",2023-04-30 20:05:00.005461-04:00
2,FRC3,14,17,72,59,1.0,False,traffic-service-flow 1.0.087,"[{'latitude': 43.646886954993356, 'longitude':...",2023-04-30 20:10:00.004552-04:00
3,FRC3,10,18,100,56,1.0,False,traffic-service-flow 1.0.087,"[{'latitude': 43.646886954993356, 'longitude':...",2023-04-30 20:15:00.009415-04:00
4,FRC3,12,18,84,56,1.0,False,traffic-service-flow 1.0.087,"[{'latitude': 43.646886954993356, 'longitude':...",2023-04-30 20:20:00.002541-04:00
...,...,...,...,...,...,...,...,...,...,...
8922,FRC3,11,17,91,59,1.0,False,traffic-service-flow 1.0.088,"[{'latitude': 43.646886954993356, 'longitude':...",2023-05-31 19:35:00.003493-04:00
8923,FRC3,17,17,59,59,1.0,False,traffic-service-flow 1.0.088,"[{'latitude': 43.646886954993356, 'longitude':...",2023-05-31 19:40:00.004800-04:00
8924,FRC3,10,16,100,63,1.0,False,traffic-service-flow 1.0.088,"[{'latitude': 43.646886954993356, 'longitude':...",2023-05-31 19:45:00.007860-04:00
8925,FRC3,11,16,91,63,1.0,False,traffic-service-flow 1.0.088,"[{'latitude': 43.646886954993356, 'longitude':...",2023-05-31 19:50:00.005163-04:00


In [6]:
yonge_speed.drop(["frc", "freeFlowSpeed", "currentTravelTime", "freeFlowTravelTime", "confidence", "roadClosure", "@version", "coordinates"], axis=1, inplace=True)
yonge_speed

,currentSpeed,collectedAt
0,12,2023-04-30 20:00:00.007773-04:00
1,13,2023-04-30 20:05:00.005461-04:00
2,14,2023-04-30 20:10:00.004552-04:00
3,10,2023-04-30 20:15:00.009415-04:00
4,12,2023-04-30 20:20:00.002541-04:00
...,...,...
8922,11,2023-05-31 19:35:00.003493-04:00
8923,17,2023-05-31 19:40:00.004800-04:00
8924,10,2023-05-31 19:45:00.007860-04:00
8925,11,2023-05-31 19:50:00.005163-04:00


In [7]:
yonge_speed["collectedAt"] = pd.to_datetime(yonge_speed["collectedAt"], utc=True)
yonge_speed["collectedAt"] = yonge_speed["collectedAt"].dt.floor(freq="t")
yonge_speed

,currentSpeed,collectedAt
0,12,2023-05-01 00:00:00+00:00
1,13,2023-05-01 00:05:00+00:00
2,14,2023-05-01 00:10:00+00:00
3,10,2023-05-01 00:15:00+00:00
4,12,2023-05-01 00:20:00+00:00
...,...,...
8922,11,2023-05-31 23:35:00+00:00
8923,17,2023-05-31 23:40:00+00:00
8924,10,2023-05-31 23:45:00+00:00
8925,11,2023-05-31 23:50:00+00:00


# Process Weather Data

In [8]:
weather_data = pd.read_csv("../../csv/v2/202305_weather_data.csv.zip")
weather_data

,weather,base,visibility,datetime,timezone,id,name,Code,collectedAt,longitude,...,windGust,cloudsAll,systemType,systemId,systemCountry,systemSunrise,systemSunset,seaLevel,groundLevel,rain1h
0,[804],stations,10000,1682898981,-14400,6167863,Downtown Toronto,200,2023-04-30 20:00:00.007773-04:00,-79.3839,...,NaN,100,1,718,CA,1682849495,1682900266,NaN,NaN,NaN
1,[804],stations,10000,1682899498,-14400,6167863,Downtown Toronto,200,2023-04-30 20:05:00.005461-04:00,-79.3839,...,NaN,100,1,718,CA,1682849495,1682900266,NaN,NaN,NaN
2,[804],stations,10000,1682899498,-14400,6167863,Downtown Toronto,200,2023-04-30 20:10:00.004552-04:00,-79.3839,...,NaN,100,1,718,CA,1682849495,1682900266,NaN,NaN,NaN
3,[804],stations,10000,1682899542,-14400,6167863,Downtown Toronto,200,2023-04-30 20:15:00.009415-04:00,-79.3839,...,NaN,100,1,718,CA,1682849495,1682900266,NaN,NaN,NaN
4,[804],stations,10000,1682900232,-14400,6167863,Downtown Toronto,200,2023-04-30 20:20:00.002541-04:00,-79.3809,...,NaN,100,1,718,CA,1682849494,1682900265,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8922,[802],stations,10000,1685575417,-14400,6167863,Downtown Toronto,200,2023-05-31 19:35:00.003493-04:00,-79.3839,...,1.79,32,1,718,CA,1685525985,1685580659,NaN,NaN,NaN
8923,[800],stations,10000,1685575868,-14400,6167863,Downtown Toronto,200,2023-05-31 19:40:00.004800-04:00,-79.3839,...,NaN,0,1,718,CA,1685525985,1685580659,NaN,NaN,NaN
8924,[800],stations,10000,1685576160,-14400,6167863,Downtown Toronto,200,2023-05-31 19:45:00.007860-04:00,-79.3839,...,NaN,0,1,718,CA,1685525985,1685580659,NaN,NaN,NaN
8925,[800],stations,10000,1685576646,-14400,6167863,Downtown Toronto,200,2023-05-31 19:50:00.005163-04:00,-79.3809,...,NaN,0,1,718,CA,1685525985,1685580657,NaN,NaN,NaN


In [9]:
weather_data["weather"] = weather_data["weather"].apply(lambda x: ast.literal_eval(x)[0])

weather_data["collectedAt"] = pd.to_datetime(weather_data["collectedAt"], utc=True)
weather_data["collectedAt"] = weather_data["collectedAt"].apply(lambda x: x.to_datetime64())
weather_data["collectedAt"] = weather_data.collectedAt.dt.floor(freq="t")

weather_data["systemSunrise"] = weather_data["systemSunrise"] + weather_data["timezone"]
weather_data["systemSunrise"] = pd.to_datetime(weather_data["systemSunrise"], unit="s")
weather_data["systemSunrise"] = weather_data["systemSunrise"].apply(lambda x: x.to_datetime64())

weather_data["systemSunset"] = weather_data["systemSunset"] + weather_data["timezone"]
weather_data["systemSunset"] = pd.to_datetime(weather_data["systemSunset"], unit="s")
weather_data["systemSunset"] = weather_data["systemSunset"].apply(lambda x: x.to_datetime64())

weather_data = weather_data.drop(
    columns=["base",
             "datetime",
             "timezone",
             "id",
             "name",
             "Code",
             "longitude",
             "latitude",
             "tempMin",
             "tempMax",
             "pressure",
             "windGust",
             "systemType",
             "systemId",
             "systemCountry",
             "systemSunrise",
             "systemSunset",
             "seaLevel",
             "groundLevel"])

weather_data = weather_data.fillna(value=0, axis=1)

weather_data

,weather,visibility,collectedAt,temperature,FeelsLike,humidity,windSpeed,windDegree,cloudsAll,rain1h
0,804,10000,2023-05-01 00:00:00,9.19,6.73,90,4.63,290,100,0
1,804,10000,2023-05-01 00:05:00,9.12,6.64,90,4.63,290,100,0
2,804,10000,2023-05-01 00:10:00,9.12,6.64,90,4.63,290,100,0
3,804,10000,2023-05-01 00:15:00,9.13,6.65,90,4.63,290,100,0
4,804,10000,2023-05-01 00:20:00,9.24,6.22,89,6.17,300,100,0
...,...,...,...,...,...,...,...,...,...,...
8922,802,10000,2023-05-31 23:35:00,28.05,27.51,37,0.89,119,32,0
8923,800,10000,2023-05-31 23:40:00,27.17,27.04,41,1.54,260,0,0
8924,800,10000,2023-05-31 23:45:00,27.12,27.0,41,1.54,260,0,0
8925,800,10000,2023-05-31 23:50:00,26.77,26.79,42,1.54,260,0,0


In [10]:
weather_data["snow1h"] = 0
weather_data

,weather,visibility,collectedAt,temperature,FeelsLike,humidity,windSpeed,windDegree,cloudsAll,rain1h,snow1h
0,804,10000,2023-05-01 00:00:00,9.19,6.73,90,4.63,290,100,0,0
1,804,10000,2023-05-01 00:05:00,9.12,6.64,90,4.63,290,100,0,0
2,804,10000,2023-05-01 00:10:00,9.12,6.64,90,4.63,290,100,0,0
3,804,10000,2023-05-01 00:15:00,9.13,6.65,90,4.63,290,100,0,0
4,804,10000,2023-05-01 00:20:00,9.24,6.22,89,6.17,300,100,0,0
...,...,...,...,...,...,...,...,...,...,...,...
8922,802,10000,2023-05-31 23:35:00,28.05,27.51,37,0.89,119,32,0,0
8923,800,10000,2023-05-31 23:40:00,27.17,27.04,41,1.54,260,0,0,0
8924,800,10000,2023-05-31 23:45:00,27.12,27.0,41,1.54,260,0,0,0
8925,800,10000,2023-05-31 23:50:00,26.77,26.79,42,1.54,260,0,0,0


In [11]:
rainy_condition_codes = np.concatenate([np.arange(200,300,1),np.arange(300,400,1),np.arange(500,600,1)])

weather_data["cloudy"] = weather_data.apply(lambda x: 1 if 801 <= x["weather"] <= 804 else 0,axis=1)
weather_data["clear"] = weather_data.apply(lambda x: 1 if x["weather"] == 800 else 0,axis=1)
weather_data["atmosphere"] = weather_data.apply(lambda x: 1 if 700 <= x["weather"] <= 799 else 0,axis=1)
weather_data["snowy"] = weather_data.apply(lambda x: 1 if 600 <= x["weather"] <= 699 else 0,axis=1)
weather_data["rainy"] = weather_data.apply(lambda x: 1 if x["weather"] in rainy_condition_codes else 0,axis=1)

weather_data.drop("weather",inplace=True,axis=1)

weather_data

,visibility,collectedAt,temperature,FeelsLike,humidity,windSpeed,windDegree,cloudsAll,rain1h,snow1h,cloudy,clear,atmosphere,snowy,rainy
0,10000,2023-05-01 00:00:00,9.19,6.73,90,4.63,290,100,0,0,1,0,0,0,0
1,10000,2023-05-01 00:05:00,9.12,6.64,90,4.63,290,100,0,0,1,0,0,0,0
2,10000,2023-05-01 00:10:00,9.12,6.64,90,4.63,290,100,0,0,1,0,0,0,0
3,10000,2023-05-01 00:15:00,9.13,6.65,90,4.63,290,100,0,0,1,0,0,0,0
4,10000,2023-05-01 00:20:00,9.24,6.22,89,6.17,300,100,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8922,10000,2023-05-31 23:35:00,28.05,27.51,37,0.89,119,32,0,0,1,0,0,0,0
8923,10000,2023-05-31 23:40:00,27.17,27.04,41,1.54,260,0,0,0,0,1,0,0,0
8924,10000,2023-05-31 23:45:00,27.12,27.0,41,1.54,260,0,0,0,0,1,0,0,0
8925,10000,2023-05-31 23:50:00,26.77,26.79,42,1.54,260,0,0,0,0,1,0,0,0


In [12]:
weather_data["visibility"] = weather_data.apply(lambda x: x["visibility"]/1000,axis=1)
weather_data

,visibility,collectedAt,temperature,FeelsLike,humidity,windSpeed,windDegree,cloudsAll,rain1h,snow1h,cloudy,clear,atmosphere,snowy,rainy
0,10.0,2023-05-01 00:00:00,9.19,6.73,90,4.63,290,100,0,0,1,0,0,0,0
1,10.0,2023-05-01 00:05:00,9.12,6.64,90,4.63,290,100,0,0,1,0,0,0,0
2,10.0,2023-05-01 00:10:00,9.12,6.64,90,4.63,290,100,0,0,1,0,0,0,0
3,10.0,2023-05-01 00:15:00,9.13,6.65,90,4.63,290,100,0,0,1,0,0,0,0
4,10.0,2023-05-01 00:20:00,9.24,6.22,89,6.17,300,100,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8922,10.0,2023-05-31 23:35:00,28.05,27.51,37,0.89,119,32,0,0,1,0,0,0,0
8923,10.0,2023-05-31 23:40:00,27.17,27.04,41,1.54,260,0,0,0,0,1,0,0,0
8924,10.0,2023-05-31 23:45:00,27.12,27.0,41,1.54,260,0,0,0,0,1,0,0,0
8925,10.0,2023-05-31 23:50:00,26.77,26.79,42,1.54,260,0,0,0,0,1,0,0,0


In [13]:
weather_data["rain1h"] = weather_data.apply(lambda x: x["rain1h"]*10,axis=1)
weather_data["snow1h"] = weather_data.apply(lambda x: x["snow1h"]*10,axis=1)
weather_data

,visibility,collectedAt,temperature,FeelsLike,humidity,windSpeed,windDegree,cloudsAll,rain1h,snow1h,cloudy,clear,atmosphere,snowy,rainy
0,10.0,2023-05-01 00:00:00,9.19,6.73,90,4.63,290,100,0.0,0,1,0,0,0,0
1,10.0,2023-05-01 00:05:00,9.12,6.64,90,4.63,290,100,0.0,0,1,0,0,0,0
2,10.0,2023-05-01 00:10:00,9.12,6.64,90,4.63,290,100,0.0,0,1,0,0,0,0
3,10.0,2023-05-01 00:15:00,9.13,6.65,90,4.63,290,100,0.0,0,1,0,0,0,0
4,10.0,2023-05-01 00:20:00,9.24,6.22,89,6.17,300,100,0.0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8922,10.0,2023-05-31 23:35:00,28.05,27.51,37,0.89,119,32,0.0,0,1,0,0,0,0
8923,10.0,2023-05-31 23:40:00,27.17,27.04,41,1.54,260,0,0.0,0,0,1,0,0,0
8924,10.0,2023-05-31 23:45:00,27.12,27.0,41,1.54,260,0,0.0,0,0,1,0,0,0
8925,10.0,2023-05-31 23:50:00,26.77,26.79,42,1.54,260,0,0.0,0,0,1,0,0,0


# Process Incident Data

In [14]:
incidents_data = pd.read_csv("../../csv/v2/202305_incidents_data.csv.zip")
incidents_data

,type,id,iconCategory,magnitudeOfDelay,startTime,endTime,from,to,length,delay,...,events,tmc,geometryType,geometryCoordinates,collectedAt,tmcCountryCode,tmcTableNumber,tmcTableVersion,tmcDirection,tmcPoints
0,Feature,ca23810f698efbea5640ebecaa00455e,8.0,4.0,2023-04-29T05:12:03Z,2023-05-01T03:59:00Z,Wellington St E,King St E / Victoria St,78.646010,NaN,...,"[{'code': 401, 'description': 'Closed', 'iconC...",NaN,LineString,"[[-79.3760370322, 43.6481183789], [-79.3762435...",2023-04-30 23:15:00.004397-04:00,NaN,NaN,NaN,NaN,NaN
1,Feature,379b88faa310aa3d36b258a380a73949,9.0,0.0,2023-04-30T18:40:00Z,NaN,Dunn Ave/Jameson Ave (ON-2),ON-11A/Lower Simcoe St/Yonge St (ON-2),1653.060000,NaN,...,"[{'code': 701, 'description': 'Roadworks', 'ic...",NaN,LineString,"[[-79.3992287525, 43.6378442222], [-79.3983637...",2023-04-30 23:15:00.004397-04:00,C,9.0,9.0,positive,"[{'location': 4312, 'offset': 646}, {'location..."
2,Feature,3966ee76c3c3241e1f083e45a3098096,6.0,1.0,2023-05-01T10:04:00Z,2023-05-01T10:41:30Z,Dunn Ave/Jameson Ave (ON-2),ON-11A/Lower Simcoe St/Yonge St (ON-2),1375.454517,68.0,...,"[{'code': 115, 'description': 'Slow traffic', ...",NaN,LineString,"[[-79.3880130955, 43.6393677087], [-79.3877891...",2023-05-01 06:20:00.010109-04:00,C,9.0,9.0,positive,"[{'location': 4312, 'offset': 0}, {'location':..."
3,Feature,ca92bbb7ced5f91daaad5bcd0324d0f2,6.0,2.0,2023-05-01T11:33:30Z,2023-05-01T11:45:30Z,Don Valley Pkwy (ON-2),ON-11A/York St/Bay St (ON-2),389.701176,33.0,...,"[{'code': 108, 'description': 'Backed up traff...",NaN,LineString,"[[-79.3727888771, 43.6452041577], [-79.3735533...",2023-05-01 07:35:00.003947-04:00,C,9.0,9.0,negative,"[{'location': 4311, 'offset': 432}, {'location..."
4,Feature,496cc53347a6155a684fec5c04b6b92c,6.0,1.0,2023-05-01T11:47:30Z,2023-05-01T12:08:30Z,Front St W (Bay St),Richmond St W (Bay St),445.277000,136.0,...,"[{'code': 115, 'description': 'Slow traffic', ...",NaN,LineString,"[[-79.3796767898, 43.6473311868], [-79.3798497...",2023-05-01 07:50:00.005826-04:00,C,9.0,9.0,positive,"[{'location': 12206, 'offset': 0}, {'location'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2372,Feature,5719c3022810dc93dbd0245f6843b6e5,6.0,2.0,2023-05-31T17:40:00Z,2023-05-31T23:37:30Z,King St W (University Ave/ON-11A),Gardiner Expy/Lake Shore Blvd (ON-11A),665.436720,348.0,...,"[{'code': 108, 'description': 'Backed up traff...",NaN,LineString,"[[-79.3844122298, 43.6469543572], [-79.3840823...",2023-05-31 19:15:00.007010-04:00,C,9.0,9.0,negative,"[{'location': 4588, 'offset': 90}, {'location'..."
2373,Feature,d56e8036f4bc0b6b020d9fb9c5753fe1,6.0,3.0,2023-05-31T23:06:30Z,2023-05-31T23:41:30Z,Station St (Lower Simcoe St),Queens Quay W (Lower Simcoe St),252.883000,250.0,...,"[{'code': 101, 'description': 'Stopped traffic...",NaN,LineString,"[[-79.3842284985, 43.6428170209], [-79.3841855...",2023-05-31 19:15:00.007010-04:00,C,9.0,9.0,negative,"[{'location': 12591, 'offset': 126}, {'locatio..."
2374,Feature,62a2cc6104bc823ca307ce174d7946a8,6.0,1.0,2023-05-31T23:21:00Z,2023-05-31T23:44:30Z,Lower Jarvis St (Queens Quay),Lower Spadina Ave (Queens Quay),1003.305708,170.0,...,"[{'code': 115, 'description': 'Slow traffic', ...",NaN,LineString,"[[-79.3713498719, 43.6430745058], [-79.3718970...",2023-05-31 19:25:00.003575-04:00,C,9.0,9.0,negative,"[{'location': 12166, 'offset': 771}, {'locatio..."
2375,Feature,4c027e5fc8d4751a1cbc2adae8876dfc,6.0,3.0,2023-05-31T22:36:00Z,2023-05-31T23:49:30Z,Cherry St (Lake Shore Blvd/Harbour St),ON-11/Yonge St (Lake Shore Blvd/Harbour St),1239.728312,762.0,...,"[{'code': 101, 'description': 'Stopped traffic...",NaN,LineString,"[[-79.3590921767, 43.6486306942], [-79.3591297...",2023-05-31 19:30:00.003046-04:00,C,9.0,9.0,negative,"[{'location': 4849, 'offset': 204}, {'location..."


In [15]:
incidents_data.drop(["id","tmcCountryCode", "tmcTableNumber", "tmcTableVersion", "tmcPoints", "geometryType", "tmc"],inplace=True,axis=1)
incidents_data

,type,iconCategory,magnitudeOfDelay,startTime,endTime,from,to,length,delay,roadNumbers,timeValidity,probabilityOfOccurrence,numberOfReports,lastReportTime,events,geometryCoordinates,collectedAt,tmcDirection
0,Feature,8.0,4.0,2023-04-29T05:12:03Z,2023-05-01T03:59:00Z,Wellington St E,King St E / Victoria St,78.646010,NaN,[],present,certain,NaN,NaN,"[{'code': 401, 'description': 'Closed', 'iconC...","[[-79.3760370322, 43.6481183789], [-79.3762435...",2023-04-30 23:15:00.004397-04:00,NaN
1,Feature,9.0,0.0,2023-04-30T18:40:00Z,NaN,Dunn Ave/Jameson Ave (ON-2),ON-11A/Lower Simcoe St/Yonge St (ON-2),1653.060000,NaN,['Gardiner Expy E'],present,probable,NaN,NaN,"[{'code': 701, 'description': 'Roadworks', 'ic...","[[-79.3992287525, 43.6378442222], [-79.3983637...",2023-04-30 23:15:00.004397-04:00,positive
2,Feature,6.0,1.0,2023-05-01T10:04:00Z,2023-05-01T10:41:30Z,Dunn Ave/Jameson Ave (ON-2),ON-11A/Lower Simcoe St/Yonge St (ON-2),1375.454517,68.0,['Gardiner Expy E'],present,certain,NaN,NaN,"[{'code': 115, 'description': 'Slow traffic', ...","[[-79.3880130955, 43.6393677087], [-79.3877891...",2023-05-01 06:20:00.010109-04:00,positive
3,Feature,6.0,2.0,2023-05-01T11:33:30Z,2023-05-01T11:45:30Z,Don Valley Pkwy (ON-2),ON-11A/York St/Bay St (ON-2),389.701176,33.0,['Gardiner Expy W'],present,certain,NaN,NaN,"[{'code': 108, 'description': 'Backed up traff...","[[-79.3727888771, 43.6452041577], [-79.3735533...",2023-05-01 07:35:00.003947-04:00,negative
4,Feature,6.0,1.0,2023-05-01T11:47:30Z,2023-05-01T12:08:30Z,Front St W (Bay St),Richmond St W (Bay St),445.277000,136.0,[],present,certain,NaN,NaN,"[{'code': 115, 'description': 'Slow traffic', ...","[[-79.3796767898, 43.6473311868], [-79.3798497...",2023-05-01 07:50:00.005826-04:00,positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2372,Feature,6.0,2.0,2023-05-31T17:40:00Z,2023-05-31T23:37:30Z,King St W (University Ave/ON-11A),Gardiner Expy/Lake Shore Blvd (ON-11A),665.436720,348.0,['ON-11A'],present,certain,NaN,NaN,"[{'code': 108, 'description': 'Backed up traff...","[[-79.3844122298, 43.6469543572], [-79.3840823...",2023-05-31 19:15:00.007010-04:00,negative
2373,Feature,6.0,3.0,2023-05-31T23:06:30Z,2023-05-31T23:41:30Z,Station St (Lower Simcoe St),Queens Quay W (Lower Simcoe St),252.883000,250.0,[],present,certain,NaN,NaN,"[{'code': 101, 'description': 'Stopped traffic...","[[-79.3842284985, 43.6428170209], [-79.3841855...",2023-05-31 19:15:00.007010-04:00,negative
2374,Feature,6.0,1.0,2023-05-31T23:21:00Z,2023-05-31T23:44:30Z,Lower Jarvis St (Queens Quay),Lower Spadina Ave (Queens Quay),1003.305708,170.0,[],present,certain,NaN,NaN,"[{'code': 115, 'description': 'Slow traffic', ...","[[-79.3713498719, 43.6430745058], [-79.3718970...",2023-05-31 19:25:00.003575-04:00,negative
2375,Feature,6.0,3.0,2023-05-31T22:36:00Z,2023-05-31T23:49:30Z,Cherry St (Lake Shore Blvd/Harbour St),ON-11/Yonge St (Lake Shore Blvd/Harbour St),1239.728312,762.0,[],present,certain,NaN,NaN,"[{'code': 101, 'description': 'Stopped traffic...","[[-79.3590921767, 43.6486306942], [-79.3591297...",2023-05-31 19:30:00.003046-04:00,negative


In [16]:
incidents_data.fillna(0,inplace=True)
incidents_data

,type,iconCategory,magnitudeOfDelay,startTime,endTime,from,to,length,delay,roadNumbers,timeValidity,probabilityOfOccurrence,numberOfReports,lastReportTime,events,geometryCoordinates,collectedAt,tmcDirection
0,Feature,8.0,4.0,2023-04-29T05:12:03Z,2023-05-01T03:59:00Z,Wellington St E,King St E / Victoria St,78.646010,0.0,[],present,certain,0.0,0,"[{'code': 401, 'description': 'Closed', 'iconC...","[[-79.3760370322, 43.6481183789], [-79.3762435...",2023-04-30 23:15:00.004397-04:00,0
1,Feature,9.0,0.0,2023-04-30T18:40:00Z,0,Dunn Ave/Jameson Ave (ON-2),ON-11A/Lower Simcoe St/Yonge St (ON-2),1653.060000,0.0,['Gardiner Expy E'],present,probable,0.0,0,"[{'code': 701, 'description': 'Roadworks', 'ic...","[[-79.3992287525, 43.6378442222], [-79.3983637...",2023-04-30 23:15:00.004397-04:00,positive
2,Feature,6.0,1.0,2023-05-01T10:04:00Z,2023-05-01T10:41:30Z,Dunn Ave/Jameson Ave (ON-2),ON-11A/Lower Simcoe St/Yonge St (ON-2),1375.454517,68.0,['Gardiner Expy E'],present,certain,0.0,0,"[{'code': 115, 'description': 'Slow traffic', ...","[[-79.3880130955, 43.6393677087], [-79.3877891...",2023-05-01 06:20:00.010109-04:00,positive
3,Feature,6.0,2.0,2023-05-01T11:33:30Z,2023-05-01T11:45:30Z,Don Valley Pkwy (ON-2),ON-11A/York St/Bay St (ON-2),389.701176,33.0,['Gardiner Expy W'],present,certain,0.0,0,"[{'code': 108, 'description': 'Backed up traff...","[[-79.3727888771, 43.6452041577], [-79.3735533...",2023-05-01 07:35:00.003947-04:00,negative
4,Feature,6.0,1.0,2023-05-01T11:47:30Z,2023-05-01T12:08:30Z,Front St W (Bay St),Richmond St W (Bay St),445.277000,136.0,[],present,certain,0.0,0,"[{'code': 115, 'description': 'Slow traffic', ...","[[-79.3796767898, 43.6473311868], [-79.3798497...",2023-05-01 07:50:00.005826-04:00,positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2372,Feature,6.0,2.0,2023-05-31T17:40:00Z,2023-05-31T23:37:30Z,King St W (University Ave/ON-11A),Gardiner Expy/Lake Shore Blvd (ON-11A),665.436720,348.0,['ON-11A'],present,certain,0.0,0,"[{'code': 108, 'description': 'Backed up traff...","[[-79.3844122298, 43.6469543572], [-79.3840823...",2023-05-31 19:15:00.007010-04:00,negative
2373,Feature,6.0,3.0,2023-05-31T23:06:30Z,2023-05-31T23:41:30Z,Station St (Lower Simcoe St),Queens Quay W (Lower Simcoe St),252.883000,250.0,[],present,certain,0.0,0,"[{'code': 101, 'description': 'Stopped traffic...","[[-79.3842284985, 43.6428170209], [-79.3841855...",2023-05-31 19:15:00.007010-04:00,negative
2374,Feature,6.0,1.0,2023-05-31T23:21:00Z,2023-05-31T23:44:30Z,Lower Jarvis St (Queens Quay),Lower Spadina Ave (Queens Quay),1003.305708,170.0,[],present,certain,0.0,0,"[{'code': 115, 'description': 'Slow traffic', ...","[[-79.3713498719, 43.6430745058], [-79.3718970...",2023-05-31 19:25:00.003575-04:00,negative
2375,Feature,6.0,3.0,2023-05-31T22:36:00Z,2023-05-31T23:49:30Z,Cherry St (Lake Shore Blvd/Harbour St),ON-11/Yonge St (Lake Shore Blvd/Harbour St),1239.728312,762.0,[],present,certain,0.0,0,"[{'code': 101, 'description': 'Stopped traffic...","[[-79.3590921767, 43.6486306942], [-79.3591297...",2023-05-31 19:30:00.003046-04:00,negative


In [17]:
incidents_data["geometryCoordinates"] = incidents_data["geometryCoordinates"].progress_apply(lambda x: [y[::-1] for y in ast.literal_eval(x)])
incidents_data

100%|█████████████████████████████████████| 2377/2377 [00:00<00:00, 3678.84it/s]


,type,iconCategory,magnitudeOfDelay,startTime,endTime,from,to,length,delay,roadNumbers,timeValidity,probabilityOfOccurrence,numberOfReports,lastReportTime,events,geometryCoordinates,collectedAt,tmcDirection
0,Feature,8.0,4.0,2023-04-29T05:12:03Z,2023-05-01T03:59:00Z,Wellington St E,King St E / Victoria St,78.646010,0.0,[],present,certain,0.0,0,"[{'code': 401, 'description': 'Closed', 'iconC...","[[43.6481183789, -79.3760370322], [43.64856767...",2023-04-30 23:15:00.004397-04:00,0
1,Feature,9.0,0.0,2023-04-30T18:40:00Z,0,Dunn Ave/Jameson Ave (ON-2),ON-11A/Lower Simcoe St/Yonge St (ON-2),1653.060000,0.0,['Gardiner Expy E'],present,probable,0.0,0,"[{'code': 701, 'description': 'Roadworks', 'ic...","[[43.6378442222, -79.3992287525], [43.63781601...",2023-04-30 23:15:00.004397-04:00,positive
2,Feature,6.0,1.0,2023-05-01T10:04:00Z,2023-05-01T10:41:30Z,Dunn Ave/Jameson Ave (ON-2),ON-11A/Lower Simcoe St/Yonge St (ON-2),1375.454517,68.0,['Gardiner Expy E'],present,certain,0.0,0,"[{'code': 115, 'description': 'Slow traffic', ...","[[43.6393677087, -79.3880130955], [43.63940926...",2023-05-01 06:20:00.010109-04:00,positive
3,Feature,6.0,2.0,2023-05-01T11:33:30Z,2023-05-01T11:45:30Z,Don Valley Pkwy (ON-2),ON-11A/York St/Bay St (ON-2),389.701176,33.0,['Gardiner Expy W'],present,certain,0.0,0,"[{'code': 108, 'description': 'Backed up traff...","[[43.6452041577, -79.3727888771], [43.64503790...",2023-05-01 07:35:00.003947-04:00,negative
4,Feature,6.0,1.0,2023-05-01T11:47:30Z,2023-05-01T12:08:30Z,Front St W (Bay St),Richmond St W (Bay St),445.277000,136.0,[],present,certain,0.0,0,"[{'code': 115, 'description': 'Slow traffic', ...","[[43.6473311868, -79.3796767898], [43.64771341...",2023-05-01 07:50:00.005826-04:00,positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2372,Feature,6.0,2.0,2023-05-31T17:40:00Z,2023-05-31T23:37:30Z,King St W (University Ave/ON-11A),Gardiner Expy/Lake Shore Blvd (ON-11A),665.436720,348.0,['ON-11A'],present,certain,0.0,0,"[{'code': 108, 'description': 'Backed up traff...","[[43.6469543572, -79.3844122298], [43.64638434...",2023-05-31 19:15:00.007010-04:00,negative
2373,Feature,6.0,3.0,2023-05-31T23:06:30Z,2023-05-31T23:41:30Z,Station St (Lower Simcoe St),Queens Quay W (Lower Simcoe St),252.883000,250.0,[],present,certain,0.0,0,"[{'code': 101, 'description': 'Stopped traffic...","[[43.6428170209, -79.3842284985], [43.64271778...",2023-05-31 19:15:00.007010-04:00,negative
2374,Feature,6.0,1.0,2023-05-31T23:21:00Z,2023-05-31T23:44:30Z,Lower Jarvis St (Queens Quay),Lower Spadina Ave (Queens Quay),1003.305708,170.0,[],present,certain,0.0,0,"[{'code': 115, 'description': 'Slow traffic', ...","[[43.6430745058, -79.3713498719], [43.64288137...",2023-05-31 19:25:00.003575-04:00,negative
2375,Feature,6.0,3.0,2023-05-31T22:36:00Z,2023-05-31T23:49:30Z,Cherry St (Lake Shore Blvd/Harbour St),ON-11/Yonge St (Lake Shore Blvd/Harbour St),1239.728312,762.0,[],present,certain,0.0,0,"[{'code': 101, 'description': 'Stopped traffic...","[[43.6486306942, -79.3590921767], [43.64861862...",2023-05-31 19:30:00.003046-04:00,negative


In [18]:
yonge_detectors = pd.read_csv("../../csv/v2/030_yonge_count_processed.csv.zip")
gardiner_detectors = pd.read_csv("../../csv/v2/040_gardiner_count_processed.csv.zip")

yonge_detectors.drop_duplicates("location",inplace=True)
gardiner_detectors.drop_duplicates("detector_id",inplace=True)
yonge_detectors = yonge_detectors[yonge_detectors["location"].str.contains("YONGE ST")]

yonge_detectors["coords"] = yonge_detectors.apply(lambda x: (x.lat, x.lng), axis=1)
gardiner_detectors["coords"] = gardiner_detectors.apply(lambda x: (x.latitude, x.longitude), axis=1)

yonge_detector_data = yonge_detectors[["location","coords"]]
gardiner_detector_data = gardiner_detectors[["detector_id","coords"]]

In [19]:
import os

def calc_dist(a,b):
    return geodesic(a,b).km

calc_dist_vectorized = np.vectorize(calc_dist,signature='(x),(y)->()')

if "202305_tmp_incidents.csv.zip" not in os.listdir("./data"):
    def check_all_coords_yonge(incident_coords):
        for idx, yonge_detector_data_row in yonge_detector_data.iterrows():
            distances = calc_dist_vectorized(incident_coords, yonge_detector_data_row["coords"])
            if np.any(distances <= 0.50):
                return yonge_detector_data_row["location"]
        return "FAR"

    def check_all_coords_gardiner(incident_coords):
        for idx, gardiner_detector_data_row in gardiner_detector_data.iterrows():
            distances = calc_dist_vectorized(incident_coords, gardiner_detector_data_row["coords"])
            if np.any(distances <= 0.50):
                return gardiner_detector_data_row["detector_id"]
        return "FAR"

    incidents_data['close_to_yonge_detector'] = incidents_data.progress_apply(lambda x: check_all_coords_yonge(x['geometryCoordinates']),axis=1)
    incidents_data['close_to_gardiner_detector'] = incidents_data.progress_apply(lambda x: check_all_coords_gardiner(x['geometryCoordinates']),axis=1)

    incidents_data.to_csv("./data/202305_tmp_incidents.csv.zip",compression="zip",index=False)
else:
    incidents_data = pd.read_csv("./data/202305_tmp_incidents.csv.zip")

100%|███████████████████████████████████████| 2377/2377 [02:52<00:00, 13.76it/s]


In [20]:
incidents_data.drop(["type", "startTime", "endTime", "timeValidity", "numberOfReports", "lastReportTime", "roadNumbers", "probabilityOfOccurrence", "tmcDirection", "geometryCoordinates"], inplace=True, axis=1)

# Store 202305 data sets

In [21]:
gardiner_speed.to_csv("./data/202305_gardiner_flow_data.csv.zip", index=False, compression="zip")
yonge_speed.to_csv("./data/202305_yonge_flow_data.csv.zip", index=False, compression="zip")
weather_data.to_csv("./data/202305_weather_data.csv.zip", index=False, compression="zip")
incidents_data.to_csv("./data/202305_incidents_data.csv.zip", index=False, compression="zip")